In [1]:
from dotenv import load_dotenv
import os
load_dotenv(override=True) # take environment variables from .env.
ai_foundry_endpoint = os.getenv("AI_FOUNDRY_ENDPOINT")
doc_agent_id = os.getenv("DOC_AGENT_ID") #Optional. Gets the agent that only has the documentation.
op_agent_id = os.getenv("OP_AGENT_ID")
router_agent_id = os.getenv("ROUTER_AGENT_ID")

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder

print("test")

project = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=ai_foundry_endpoint)

doc_agent = project.agents.get_agent(doc_agent_id)
op_agent = project.agents.get_agent(op_agent_id)
router_agent = project.agents.get_agent(router_agent_id)

test


In [24]:
thread = project.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_LYHlP0xacdzOqnT9qRYVFlUW


In [4]:
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How to deploy pods?"
)

In [5]:
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=doc_agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

MessageRole.USER: How to deploy pods?
MessageRole.AGENT: To deploy pods in Kubernetes, you typically use a `Pod` manifest. You can create a Pod directly, but it is more common to deploy Pods as part of a higher-level abstraction like a Deployment or a ReplicaSet. Here’s how you can proceed using both methods:

### Using a Standalone Pod Manifest

1. **Create a Pod Manifest**: You will create a YAML file for your Pod. Here’s an example:

    ```yaml
    apiVersion: v1
    kind: Pod
    metadata:
      name: my-pod
    spec:
      containers:
      - name: my-container
        image: nginx
    ```

2. **Deploy the Pod**: Use the following command to create the Pod:

    ```bash
    kubectl apply -f my-pod.yaml
    ```

### Using a Deployment

Using a Deployment is often preferred because it provides updates and rollback capabilities.

1. **Create a Deployment Manifest**: Here’s an example of a Deployment YAML file:

    ```yaml
    apiVersion: apps/v1
    kind: Deployment
    metadata:
 

In [6]:
def list_all_threads():
    """
    Lists all active conversation threads for the project.
    """
    print("Listing all active threads...")
    try:
        threads = project.agents.threads.list()
        if not threads:
            print("No active threads found.")
        else:
            for thread in threads:
                print(f" - Thread ID: {thread.id}")
        print("-" * 20)
    except Exception as e:
        print(f"An error occurred while listing threads: {e}")

In [7]:
list_all_threads()

Listing all active threads...
 - Thread ID: thread_s6Z24xU2dY7Agm1U3aVl3rzA
 - Thread ID: thread_YNWORgD0anm8X8VUprPZy0Tj
 - Thread ID: thread_cSXGP5fyrRU9SGvMNJyRBkqG
 - Thread ID: thread_4C8L8NS19bWDGMch4Jo6Gzzj
 - Thread ID: thread_Chi9hOmf86WX7bMclxO4GVsP
 - Thread ID: thread_TFlW5mZS5dvhINrsFsLzwloM
 - Thread ID: thread_rBiU5im6NGhahZV9sb5x34Gi
 - Thread ID: thread_J56iaNIqWo0XRwAmv1QXY8jA
 - Thread ID: thread_Pm55S3YZswl4OWZNePiaptXh
 - Thread ID: thread_qWTn08bgy831JIhLxqczA1wc
 - Thread ID: thread_FaKlpCpFAH1aLylzPyALIdA7
 - Thread ID: thread_VydIcJDKM6LGKYkV8Mux7L8X
 - Thread ID: thread_B8U7wowzAbRckLQjpuCn1J0D
 - Thread ID: thread_bGlbbqhp8pj4mLTEM91XGTWI
 - Thread ID: thread_PBeYNabhMpC9d9q1rA81juv6
 - Thread ID: thread_5HmPXh1hvHRXzKL4UyH8yzQc
--------------------


In [8]:
def delete_all_threads():
    print("Deleting all active threads...")
    try:
        threads = project.agents.threads.list()
        if not threads:
            print("No active threads found.")
        else:
            for thread in threads:
                if thread.id != "thread_5HmPXh1hvHRXzKL4UyH8yzQc":
                    project.agents.threads.delete(thread.id)
        print("-" * 20)
    except Exception as e:
        print(f"An error occurred while deleting threads: {e}")

In [9]:
delete_all_threads()

Deleting all active threads...
--------------------


# Add tools

In [19]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from app import list_kubernetes_pods, restart_all_pods

op_agent.tools = [list_kubernetes_pods, restart_all_pods]
op_agent.update(op_agent)

for tool in op_agent.tools:
    print(f"Tool name: {tool.name}")
    print(f"Description: {tool.description}")
    print("---")

Tool name: list_kubernetes_pods
Description: A tool to list all pods in a given Kubernetes namespace (default: "default"). ONLY use when the user is clearly requesting an action like 'list pods' or 'please list pods'. DO NOT use when the user is asking for an explanation or instructions


Args:
    namespace (str): The namespace to list pods from. Defaults to "default".

Returns:
    str: A formatted list of pod names and statuses, or an error message.
---
Tool name: restart_all_pods
Description: Restart all pods in a given Kubernetes namespace. ONLY use when the user is clearly requesting an action like 'restart pods'.

Args:
    namespace (str): Namespace in which to restart all pods. Defaults to "default".
    confirm (bool): If True, actually deletes the pods. If False, just previews which pods would be restarted.

Returns:
    str: Preview of pods to be restarted, or success/error message.
---


In [23]:
# The decorator returns a Tool instance
tool_instance = list_kubernetes_pods

# Invoke it directly
result = tool_instance.invoke("What's all the pods in the environment?")
print(result)

Kubernetes configuration not found. Please ensure your ~/.kube/config file is set up correctly.


In [7]:
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's all the pods in the default namespace?"
)

In [8]:
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=op_agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

MessageRole.USER: What's all the pods in the default namespace?
MessageRole.AGENT: It seems I'm unable to access the tools needed to provide you the information about pods in the default namespace. Please try again later or use a different method to retrieve that information.


# Router Agent

In [13]:
from dotenv import load_dotenv
import os
load_dotenv(override=True) # take environment variables from .env.
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential

In [20]:
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What do you do?"
)

In [21]:
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=router_agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

MessageRole.USER: What do you do?
MessageRole.AGENT: I analyze your requests to determine whether you're asking for an action or looking for information. Based on that, I route your query to the appropriate agent: the OperationsAssistant for actions and the DocsAssistant for information or instructions.


In [25]:
# Define a list of test queries with different intents
test_queries = [
    "How do I restart a Kubernetes pod?", # Documentation intent
    "Can you restart all the pods?",      # Operations intent
    "What is a Kubernetes deployment?",    # Documentation intent
    "What are the pods in default namespace?"   # Operations intent
]

# 4. Loop through queries and invoke the router agent
for query in test_queries:
    print(f"--- USER QUERY ---")
    print(f"  Query: '{query}'")
    print("--- AGENT RESPONSE ---")

    message = project.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=query
    )
    
    run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=router_agent.id)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    else:
        messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

        for message in messages:
            if message.text_messages:
                print(f"{message.role}: {message.text_messages[-1].text.value}")

--- USER QUERY ---
  Query: 'How do I restart a Kubernetes pod?'
--- AGENT RESPONSE ---
MessageRole.USER: How do I restart a Kubernetes pod?
MessageRole.AGENT: To restart a Kubernetes pod, you generally don't "restart" a pod directly. Instead, you can delete the pod, and Kubernetes will automatically create a new instance of the pod based on its configuration. Here are the steps to do this:

### Restarting a Pod Steps

1. **Identify the Pod Name:**
   Use the following command to list the pods in your namespace:
   ```bash
   kubectl get pods
   ```

2. **Delete the Pod:**
   Delete the pod using the command below, replacing `<pod-name>` with the name of your pod:
   ```bash
   kubectl delete pod <pod-name>
   ```

3. **Kubernetes Automatically Restarts the Pod:**
   If the pod is managed by a Deployment or ReplicaSet, Kubernetes will automatically create a new pod to replace the deleted one.

### Example
```bash
# List pods to find the pod name
kubectl get pods

# Delete the pod
kubec

# Langgraph

In [26]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [31]:
open_ai_api = os.getenv("OPEN_AI_API")
open_ai_endpoint = os.getenv("OPEN_AI_ENDPOINT")
open_ai_version = "2024-07-18"

In [34]:
# Create the router LLM
router_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=open_ai_api)

In [33]:
# Updated imports for aio and async operations
from azure.ai.agents.aio.sessions import SessionsClient
from azure.identity.aio import DefaultAzureCredential
import asyncio

# Your original invocation logic, now inside a standard function
async def call_azure_agent(agent_id: str, query: str) -> str:
    """
    Invokes an AI Foundry agent with a given query.
    """
    try:
        project_endpoint = os.environ["AZURE_AI_FOUNDRY_ENDPOINT"]
        credential = DefaultAzureCredential()
        sessions_client = SessionsClient(endpoint=project_endpoint, credential=credential)
    except KeyError as e:
        return f"Error: Environment variable {e} not set."

    try:
        async with sessions_client.create_or_get(agent_id=agent_id) as session:
            response = await session.invoke(input=query)
            return response.output
    except Exception as e:
        return f"An error occurred while invoking the agent: {e}"

# Your tool definitions, now correctly structured for LangGraph
@tool
def invoke_foundry_op_agent(query: str):
    """
    This tool invokes the AI Foundry Operations Agent.
    Use this for queries that ask to perform an action.
    """
    op_agent_id = op_agent_id
    return asyncio.run(call_azure_agent(op_agent_id, query))

@tool
def invoke_foundry_doc_agent(query: str):
    """
    This tool invokes the AI Foundry Documentation Agent.
    Use this for informational queries.
    """
    doc_agent_id = doc_agent_id
    return asyncio.run(call_azure_agent(doc_agent_id, query))

# These are the tools that will be passed to your main router LLM
tools = [invoke_foundry_op_agent, invoke_foundry_doc_agent]

ModuleNotFoundError: No module named 'azure.ai.agents.aio.sessions'

In [ ]:
# Build the graph
workflow = StateGraph(AgentState)

# Add your router as the main entry point from the start
workflow.add_node("doc_agent", invoke_foundry_doc_agent)
workflow.add_node("operational_agent", invoke_foundry_op_agent)